In [9]:
# basic libraries
import itertools
import re
import string

# data processing 
import collections
import numpy as np
import pandas as pd

# natural language pre-processing 
from textblob import TextBlob
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer

# ML pipelines
from transformers import pipeline

# sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# data visualization
import seaborn as sb
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macarthur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/macarthur/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
# read csv
df = pd.read_csv('../csv/data_clean.csv')
df_sent = df.copy()

def clean_links(df):
#replace URL of a text
    df_sent['text'] = df_sent['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

clean_links(df_sent)
df_sent['text'].head()
df_sent['text'] = df_sent['text'].astype(str)

### example cells for are testing hugging face model- remove when connected to real corpora

In [10]:
classifier = pipeline("text-classification",model='bhadresh-savani/albert-base-v2-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

Downloading: 100%|██████████| 1.11k/1.11k [00:00<00:00, 337kB/s]
Downloading: 100%|██████████| 46.8M/46.8M [00:06<00:00, 6.80MB/s]
Downloading: 100%|██████████| 1.31M/1.31M [00:00<00:00, 1.64MB/s]
Downloading: 100%|██████████| 428/428 [00:00<00:00, 210kB/s]


[[{'label': 'sadness', 'score': 0.010403607971966267}, {'label': 'joy', 'score': 0.8902178406715393}, {'label': 'love', 'score': 0.0425327830016613}, {'label': 'anger', 'score': 0.04129811003804207}, {'label': 'fear', 'score': 0.011772057972848415}, {'label': 'surprise', 'score': 0.003775620134547353}]]


In [105]:
albert_column_names = ["sadness", "joy", "love", "anger", "fear", "surprise"]
df_albert = pd.DataFrame(columns = albert_column_names)
score_sadness = []
score_joy = []
score_love = []
score_anger = []
score_fear = []
score_surprise = []
for i in df_sent['text']:
    pred = classifier(i)
    pred = pred[0]
    score_sadness.append(pred[0]['score'])
    score_joy.append(pred[1]['score'])
    score_love.append(pred[2]['score'])
    score_anger.append(pred[3]['score'])
    score_fear.append(pred[4]['score'])
    score_surprise.append(pred[5]['score'])
    # pred_l.append(pred[0]['socore'])

In [103]:
df_albert["sadness"] = score_sadness
df_albert["joy"] = score_joy
df_albert["love"] = score_love
df_albert["anger"] = score_anger
df_albert["fear"] = score_fear
df_albert["surprise"] = score_surprise

In [104]:
df_albert

,sadness,joy,love,anger,fear,surprise
0,0.070398,0.100583,0.017333,0.594864,0.203451,0.013372
1,0.055376,0.574590,0.018835,0.214630,0.120774,0.015796
